## [HW4] Hush, the seagulls are purring: Autoregressive Language Modeleling

CS 4740 + crosslists, Spring 2025

<br/>

<div align="center">
    <img src="https://media.newyorker.com/cartoons/6438b7e6a22e87ce2f4d36d5/master/w_1280,c_limit/230424_a27699.jpg" width="500"/>
    <br/>
    Source: <a href="https://www.newyorker.com/cartoons/issue-cartoons/cartoons-from-the-april-24-and-may-1-2023-issue">
"<i>To think this all began with letting autocomplete finish our sentences.</i>" by Robert Leighton</a>
    <br/>
    (Licensed from the New Yorker Cartoon Bank.)
</div>

<br/>

No part (code, documentation, comments, etc.) of this notebook or any assignment-related artefacts were generated/created, refined, or modified using generative AI tools such as ChatGPT. Cite this notebook as:
> Tushaar Gangavarapu, Darren Key<sup>&#129433;</sup>, Logan Kraver<sup>&#129433;</sup>, Lionel Tan<sup>&#129433;</sup>, Pun Chaixanien<sup>&#129436;</sup>, Kai Horstmann<sup>&#129436;</sup>, Dave Jung<sup>&#129436;</sup>, Aaishi Uppuluri<sup>&#129436;</sup>. 2023. [CS 4740 Fa'23 HW4] Hush, the seagulls are purring: On generating humorous captions from scene descriptions. GitHub. <br/>
> &nbsp;&nbsp;&nbsp;&nbsp;<sup>&#129433;</sup>equal contribution, software creators, ordered alphabetically<br/>
> &nbsp;&nbsp;&nbsp;&nbsp;<sup>&#129436;</sup>equal contribution, software testers, ordered alphabetically<br/>


---


__<font color="red">Special Instructions for this homework</font>__: This assignments builds on top of the code functionalities you already implemented in HW3. Many of the files provided for this homework are identical to those from HW3 but contain `raise NotImplementedError` in key functions. This is not a bug! These placeholders are intentional, and we expect you to copy over **your own working implementations from HW3** into these sections to complete HW4.


 <font color="red">**The milestone submission for this homework checks that all your relevant code functionalities from HW3 work as intended.**</font> You do not implement anything extra for the milestone. We will provide you code for the specific functions that are incorrect in your implementation.

**How will the milestone submission work?**

*  You will follow the instructions in this ipynb to create your milestone submission. Submit it to the milestone submission site ``hw4-milestone`` on gradescope.
* On Thursday, April 24th 11.59 p.m., we will make the autograder results visible to you. You will be able to see which, if any, of your function implementations are incorrect. Note that this time is chosen to be after the last possible to submit to ``hw3-programming``.
* You can comment (either anonymously or not) on this Ed post (https://edstem.org/us/courses/73802/discussion/6571140) to request correct code for *only* your incorrectly implemented functions. Our course staff will email the correct code to you. There is no penalty for hw4 associated with this.

Note:
* Parts 1-4 of this assignment can be completed without relying on any code implemented in HW3. Don't wait for Thursday to start your assignment!
* ``hw4-milestone`` does not check that *all* your code from HW3 is correct, only the relevant functionalities needed for this HW.

__Deadlines__

* Milestone deadline: <font color="red">April 28, 2025</font> (Monday), 11.59pm on the submission site(s).
* Assignment submission deadline: <font color="red">May 6, 2025</font> (Tuesday), 11.59pm on the submission site(s). We will accept submissions with no penalty upto <font color="red">May 8, 2025</font>, 11.59pm. You are allowed to use your remaining slip days (maximum of 2) after this point. The last day you may submit this assignment is <font color="red">May 10, 2025</font>.


__Policies.__ All the policies described on the course website are applicable as is (including the policy on academic integrity and the use of generative AI tools).

---

<a name="outline"></a>__Assignment outline__

* [[$\ast$] Attributions](#attr)
* [[0] Imports and installs!](#sec0)
* [[1] Assignment overview](#sec1)
* [[2] Milestone submission](#sec2)
* [[3] Data processing, tokenization, padding](#sec3)
* [[4] Sampling Tokens given Logits](#sec4)
* [[5] Finetuning the Seagull to "understand" humor](#sec5)
* [[6] Model Evaluation](#sec6)
* [[7] Final Submission!](#sec7)

---
<a name="attr"></a>
### [$\ast$] Attributions [↩︎](#outline)
Please use the space provided below to acknowledge (by name/source) all help you received (this includes generative AI tools); you're welcome to cite references in line when answering the "written" questions. If you wish to submit answers to the written questions in a separate file, please be sure to note your attributions there.


_Attributions (if any) go here._

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


---
<a name="sec0"></a>
### [0] Imports and installs! [↩︎](#outline)

Run the following code to install any external libraries and needed packages to run HW3 assignment. Before proceeding, be sure to run the second code cell to ensure that the installation is successful.



In [2]:

# set to location where you uploaded directory
%cd "/content/drive/MyDrive/cs4740_programming_assignments/hw4-release"
%pip install -r requirements.txt

import IPython

ipython = IPython.get_ipython()
#ipython.run_line_magic("sx", f"chmod +x scripts/*.py")

ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

/content/drive/MyDrive/cs4740_programming_assignments/hw4-release


In [3]:
from IPython.display import display

try:
    from seagull.utils.utils import success, colored
    display(success())
except ImportError:
    print("\033[31mInstallation failed, please retrace your steps ...")

Success!


Let's import a few packages and methods that are used throughout this notebook; in this notebook, you are free to import and/or install packages (a lot of the packages you may need should already be available) as you see fit. <font color="red">That said, you are __not__ allowed to modify the imports in any of the Python source files; further, please do not modify (delete lines, change method signatures, etc.) above or below the `TODO` placeholders within the Python source files.</font>

In [4]:
import os
import random
import textwrap
from collections import Counter
from itertools import chain
from time import process_time

import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
import yaml
from IPython.display import display

from seagull.data_processing.bbpe import BBPETokenizer
from seagull.data_processing.sequence_sampler import SequenceSamplingDataset
from seagull.model.components.embedding import Embedding
from seagull.data_processing.constants import SCENE_TOKEN, UNCANNY_TOKEN, CAPTION_TOKEN, END_OF_CAPTION_TOKEN
from seagull.model.components.transformer_layer import TransformerLayer
from seagull.model.heads.seagull_lm import SeagullLM, greedy_decode, top_p_decode, top_k_decode, sampling_decode, make_seagull_talk
from seagull.model.seagull_transformer import Seagull
from seagull.nn.transformer.ffn import FFN
from seagull.nn.transformer.mha import MultiHeadAttention
from seagull.utils.utils import colored, success


%matplotlib inline
%config InlineBackend.figure_format="retina"
plt.rcParams["font.size"] = 8

text_wrapper = textwrap.TextWrapper(width=120)

Same as HW3, let's set up a few filepaths: for convenience, we will redirect all the output artefacts to `artefacts` folder — this includes the processed/tokenized datasets, experimental artefacts (e.g., model checkpoints, configs, etc.), submission .zip files, and others.

In [5]:
BASE_DIR = os.path.abspath(".")

PRETRAINED_ARTEFACTS_DIR = os.path.join(BASE_DIR, "pretrained_artefacts")
ARTEFACTS_DIR = os.path.join(BASE_DIR, "artefacts")
SCRIPTS_DIR = os.path.join(BASE_DIR, "scripts")
CONFIGS_DIR = os.path.join(BASE_DIR, "scripts/configs")

Finally, set your and your partner's net IDs as a comma-separated string (e.g., "`tg352,ljl2`"); we'll use the `net_ids` variable to auto-populate any required information while making the submission. Please __don't__ include any spaces or special characters (e.g., "`<tg352>`" or "`tg352, ljl2`" will result in processing errors).

In [6]:
# Add your net IDs as a comma-separated string (e.g., "<net-id-1>,<net-id-2>").
net_ids = "aeh245"

if net_ids is None:
    raise ValueError("net-IDs not set; set them above")
print(f"{net_ids=} set")

net_ids='aeh245' set


---

<a name="sec1"></a>
### [1] Assignment overview <small>[↩︎](#outline)</small>

In this assignment, we will use Seagull, the transformer-based language model you implemented in HW3, to generate humorous captions for a given scene and uncanny description; inspired from what can be described as a natural language adaptation of the New Yorker Cartoon Caption Contest: https://www.newyorker.com/cartoons/contest. Here's an example (taken from the validation data):

> __Scene__: Two explorers are hiking through nature. They come across two subway cars in the brush. <font color="blue">← input to the model</font> <br/>
> __Uncanny description__: Subway cars wouldn't be way out in the middle of nowhere. <font color="blue">← input to the model</font> <br/>
> __Caption__: Of all the things we packed, I didn't think to bring my MetroCard. <font color="green">← to be generated by the model!</font>

Our objective is to train our Seagull to generate the caption text, hopefully a humorous one; two key things to note here: we want the model to 1) "generate" and 2) utilize the given scene and uncanny description to generate a suitable caption. For the "generate" part, let's formulate the task as an autoregressive problem: for the given sequence, predict the next token; use the predicted token to predict the next one, and so on.

For your reference, this was the Seagull model you already implemented in HW3:

<div align="center"><br/>
    <img src="https://i.imgur.com/3K75QZG.png" width=780px/>
<br/></div>

<a name="sec2"></a>
### [2] Milestone Submission <small>[↩︎](#outline)</small>

Let us make the milestone submission right away. We will let you know if all the relevant functions from hw3 are correct on Thursday, April 24, 11.59 p.m. You will be able to request correct implementations of incorrect functions (see instructions above) without penalty.

**Note: Please do not delay working on the assignment while you results of this check and the correct implementations (if needed)! You can proceed upto section 4 of this assignment without waiting to verify that your milestone code is correct.**


You will need to copy-paste the following functions from your HW3 implementations:
* `__getitem__` function of `SequenceSamplingDataset` in `seagull/data_processing/sequence_sampler.py`
* `forward()` method of the `Embedding` class `seagull/model/components/embedding.py`
* `attention()` function in `seagull/nn/transformer/mha.py`
* `masked_attention_probs()` in `seagull/nn/transformer/mha.py`
* `forward()` function of `FFN` class in `seagull/nn/transformer/ffn.py`
* `forward()` function of `TransformerLayer` in `seagull/model/components/transformer_layer.py`
* `forward()` function of `SeagullLM` in `seagull/model/heads/seagull_lm.py`

After you are done, run the cell below to make a milestone submission and check your code functinalities. The `make_submission.py` command when run with `--milestone-submission` flag creates a `milestone_submission.zip` file in `artefacts` folder, which is to be submitted on the submission site(s).  <font color="red">Caution: the script will overwrite any existing file named `milestone_submission.zip` in `artefacts` folder.</font>

The `milestone_submission.zip` is all that you will need to submit for the milestone (no need to submit anything else!).

In [7]:
submission_filepath = f"{ARTEFACTS_DIR}"

!python3 -m scripts.make_submission \
    --basepath-to-store-submission={os.path.join(ARTEFACTS_DIR, submission_filepath)} \
    --net-ids={net_ids} \
    --milestone-submission

if os.path.isfile(f"{os.path.join(ARTEFACTS_DIR, 'milestone_submission.zip')}"):
    display(success())
else:
    print(colored("Oops, something went wrong!", "red"))

submission stored at: /content/drive/MyDrive/cs4740_programming_assignments/hw4-release/artefacts/milestone_submission.zip
Success!


---

<a name="sec3"></a>
### [3] Data processing, tokenization, padding <small>[↩︎](#outline)</small>
Let's get back to our task of generating humorous captions from scene descriptions. The dataset for this task is located in the `dataset` folder and is processed in an [arrow format](https://huggingface.co/docs/datasets/about_arrow). Let's load the dataset and see what the dataset looks like:

In [8]:
dataset = datasets.load_from_disk("dataset/raw")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['scene', 'uncanny', 'caption'],
        num_rows: 79180
    })
    val: Dataset({
        features: ['scene', 'uncanny', 'caption'],
        num_rows: 10783
    })
    test: Dataset({
        features: ['scene', 'uncanny', 'id'],
        num_rows: 986
    })
})


The dataset has three fields for train/val splits—scene, uncanny, and caption; while, the test split lacks the caption entry. Our objective is to use our Seagull to caption those.

Run the cell below to inspect a few train/test instances. Upon running the cell, you'll note that in the train/val set, there are cases where one scene and uncanny description can be captioned in multiple ways. This adds to the task complexity; not only does the model have to learn to understand humor, but also that there's no one "right" answer!

In [9]:
split, sample_idxs = "train", [10, 11, 12]
print(colored(f"{split=}", attrs=["underline", "bold"]))
for sample_idx in sample_idxs:
    print(f"[{sample_idx}] scene: {dataset[split][sample_idx]['scene']}")
    print(f"[{sample_idx}] uncanny: {dataset[split][sample_idx]['uncanny']}")
    print(f"[{sample_idx}] caption: {colored(dataset[split][sample_idx]['caption'], color='blue')}\n")

split, sample_idxs = "test", [104, 105]
print(colored(f"\n{split=}", attrs=["underline", "bold"]))
for sample_idx in sample_idxs:
    print(f"[{sample_idx}] scene: {dataset[split][sample_idx]['scene']}")
    print(f"[{sample_idx}] uncanny: {dataset[split][sample_idx]['uncanny']}\n")

split='train'
[10] scene: An alligator is coming out of the floor. Two people stare at it. A waiter points at it.
[10] uncanny: There is an alligator coming out of the floor.
[10] caption: It's our way of encouraging social distancing.

[11] scene: An alligator is coming out of the floor. Two people stare at it. A waiter points at it.
[11] uncanny: There is an alligator coming out of the floor.
[11] caption: ... Technically, yes, but he identifies as a crocodile.

[12] scene: An alligator is coming out of the floor. Two people stare at it. A waiter points at it.
[12] uncanny: There is an alligator coming out of the floor.
[12] caption: Complaints go in the box behind the alligator.


split='test'
[104] scene: People are at a table playing games, reading, and working on the computer. A woman is reading a book and talking to a man.
[104] uncanny: People are doing many things at a table and are very busy.

[105] scene: Men have captured a giant. The giant is in a wide net. There is a cast

<a name="sec31"></a>
#### [3.1] Preprocessing for autoregressive modeling <small>[↩︎](#outline)</small>

Now that we have a concrete idea of what our dataset looks like, let's see what the preprocessed version of this dataset, suited for our autoregressive language modeling task, i.e. predicting the next token, looks like. Specifically, we need our model to generate captions, token-by-token, using the scene and uncanny description. A simple way of doing this to perform template processing via control codes to facilitate in-context learning.

We use the following template:
```text
<|scene|> ... <|uncanny|> ... <|caption|> ... <|endofcaption|>
```
where, `<|scene|>`, `<|uncanny|>`, `<|caption|>`, and `<|endofcaption|>` are special tokens used to facilitate controlled text generation. Don't worry if whatever is noted here seems alien; in simple words, we want the model to associate that the tokens between `<|caption|> ... <|endofcaption|>` correspond to humorous captions, associated with preceding scene tokens and uncanny tokens.

Consider the following example (taken from the training data):
> __Scene__: An alligator is coming out of the floor. Two people stare at it. A waiter points at it. <br/>
> __Uncanny__: There is an alligator coming out of the floor. <br/>
> __Caption__: It's our way of encouraging social distancing.

The corresponding template-filled text is:

> `<|scene|>` An alligator is coming out of the floor. Two people stare at it. A waiter points at it. `<|uncanny|>` There is an alligator coming out of the floor. `<|caption|>` It's our way of encouraging social distancing. `<|endofcaption|>`

We use the template-filled text to finetune our model on next token prediction. The above template holds for train/val splits where caption text is available. For test split, we use the following template instead:
```text
<|scene|> ... <|uncanny|> ... <|caption|>
```
Do you see what we're doing?—we're forcing the model to associate tokens after `<|caption|>` as associated captions, and at test/inference time, we hope that the model will start generating caption-like text after seeing the `<|caption|>` special token.

This processing step is already implemented for you and the dataset stored in ``dataset/processed``. Let's inspect a few samples to familiarize ourselves with the template.

In [10]:
processed_dataset = datasets.load_from_disk("dataset/processed")
print(processed_dataset, "\n")

sample_idx = 10  # change to view other samples
for split in ["train", "val", "test"]:
    print(colored(f"{split=}", attrs=["bold", "underline"]))
    print("text:", processed_dataset[split][sample_idx]["text"], "\n")

DatasetDict({
    train: Dataset({
        features: ['caption', 'text'],
        num_rows: 79180
    })
    val: Dataset({
        features: ['caption', 'text'],
        num_rows: 10783
    })
    test: Dataset({
        features: ['id', 'text', 'caption'],
        num_rows: 986
    })
}) 

split='train'
text: <|scene|> Two knights are standing guard in a throne room. one is in a hammock put up between two pillars. <|uncanny|> A guard shouldn't have a hammock where he's supposed to be standing guard. <|caption|> Going union was the best decision we ever made. <|endofcaption|> 

split='val'
text: <|scene|> A dolphin is casually riding a skateboard at a skatepark, performing impressive tricks while the crowd cheers. <|uncanny|> Dolphins are marine creatures and cannot ride skateboards due to their lack of limbs. <|caption|> Move aside, Tony Hawk, dolphins are here to show off their fin-tastic skills! <|endofcaption|> 

split='test'
text: <|scene|> A centaur man is looking at a naked man

Now that we've preprocessed our dataset in a format suited to our task of next token prediction, we can proceed to tokenize the template-filled text. We will use the same Byte-level Byte-Pair Encoding (BBPE) tokenizer  as HW3. Let's load the tokenizer; pay attention to the `special_tokens` in the tokenizer. We've already noted `<|scene|>`, `<|uncanny|>`, `<|caption|>`, `<|endofcaption|>` special tokens; other special tokens include: `<|pad|>` for padding token, `<|unk|>` for unknown token, and `<|endoftext|>` to indicate the end of text, i.e., the model has completed the ongoing chain of thought.

In [11]:
bbpe_tokenizer = BBPETokenizer()
bbpe_tokenizer.from_file(os.path.join(PRETRAINED_ARTEFACTS_DIR, "tokenizer"))
bbpe_tokenizer

BBPETokenizer(name=seagull-bbpe, vocab_size=33264, lowercase=False, punct_behavior=contiguous, special_tokens={'<|pad|>': 0, '<|unk|>': 1, '<|endoftext|>': 2, '<|scene|>': 3, '<|uncanny|>': 4, '<|caption|>': 5, '<|endofcaption|>': 6})

---
<font color="orange">__Q1.__ While masking the attention matrix using the padding mask, we only mask the pad keys and not pad queries; take a moment to convince yourself that this is true. Why is it not needed to mask the pad queries? Would not padding queries have any unintended consequences? Explain in under three sentences; unclear or misleading arguments will receive no credit.<br/>

_Hint. Think of how the cross-entropy loss is backpropagated from outputs to input tokens, specifically the pad token._
</font>

__Answer.__

As said in homework 3, assuming we only have padded tokens at the end, we don't need to mask pad queries because during backpropagation, we will ignore the output of padded query tokens with ignore_index (talked about on Ed, linked below).
However, it would have consequences. The input embedding to our attention mechanism still contains positional information since we combined both token embedding and positional embedding. If we also masked pad tokens of our queries, we would potentially lose some of this positional information. ED LINK: #634 https://edstem.org/us/courses/73802/discussion/6528460?comment=15133746

You can run the `generate_tokenized_dataset.py` command to generate the tokenized dataset (from the processed dataset) using the pretrained tokenizer.

In [12]:
!python3 -m scripts.generate_tokenized_dataset \
    --config-path={os.path.join(CONFIGS_DIR, "generate_tokenized_dataset.yml")} \
    --basepath-to-processed-dataset="dataset/processed" \
    --tokenizer-basepath={os.path.join(PRETRAINED_ARTEFACTS_DIR, "tokenizer")} \
    --path-to-store-tokenized-dataset={os.path.join(ARTEFACTS_DIR, "dataset/tokenized_dataset")}

dataset/processed
Map: 100% 79180/79180 [00:24<00:00, 3199.18 examples/s]
Map: 100% 10783/10783 [00:02<00:00, 3919.54 examples/s]
Map: 100% 986/986 [00:00<00:00, 4752.23 examples/s]
Saving the dataset (1/1 shards): 100% 79180/79180 [00:00<00:00, 138381.40 examples/s]
Saving the dataset (1/1 shards): 100% 10783/10783 [00:01<00:00, 8711.66 examples/s]
Saving the dataset (1/1 shards): 100% 986/986 [00:01<00:00, 834.48 examples/s]


Let's inspect the dataset

In [13]:
tokenized_dataset = datasets.load_from_disk(os.path.join(ARTEFACTS_DIR, "dataset/tokenized_dataset"))
print(tokenized_dataset, "\n")

sample_idx = 10  # change to view other samples
for split in ["train", "val", "test"]:
    print(colored(f"{split=}", attrs=["bold", "underline"]))
    print("input_ids:", tokenized_dataset[split][sample_idx]["input_ids"])
    print("attention_mask:", tokenized_dataset[split][sample_idx]["attention_mask"], "\n")

DatasetDict({
    train: Dataset({
        features: ['caption', 'text', 'input_ids', 'attention_mask'],
        num_rows: 79180
    })
    val: Dataset({
        features: ['caption', 'text', 'input_ids', 'attention_mask'],
        num_rows: 10783
    })
    test: Dataset({
        features: ['id', 'text', 'caption', 'input_ids', 'attention_mask'],
        num_rows: 986
    })
}) 

split='train'
input_ids: [3, 580, 6089, 319, 611, 2667, 298, 263, 2000, 768, 20, 569, 293, 298, 263, 2431, 1581, 485, 1682, 846, 8250, 20, 227, 4, 306, 2667, 1347, 13, 90, 405, 263, 2431, 1169, 416, 13, 89, 1988, 309, 338, 611, 2667, 20, 227, 5, 10421, 10010, 874, 279, 2136, 9757, 351, 3113, 1736, 20, 227, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Recall that the \_\_getitem\_\_ function you implemented in `SequenceSamplingDataset` in `seagull/data_processing/sequence_sampler.py` in HW3 takes these as input, and returns the following:


```python
{
  "input_ids": tensor([...]),
  "padding_mask": tensor([...]),
  "labels": tensor([...])  # = input_ids shifted left
}
```

<a name="sec4"></a>
### [4] Sampling Tokens given Logits <small>[↩︎](#outline)</small>

> <font color="orange">File to be modified: `seagull/model/heads/seagull_lm.py`.</font>

Inspect the `forward()` function (or just the documentation) of the `seagull/model/heads/seagull_lm.py` file. This function describes all the computation that happens in the transformer model. It corresponds to the seagull transformer figure at the start of the notebook. It takes the input_ids and the padding_mask (see structure above) as inputs. The forward pass through the Seagull transformer finally returns the logits and the output embeddings from the transformer.

Let's focus on the logits; particularly, how we can generate text using these logits. Using our autoregressive model, we want to generate the next token using the logits of the current time step. Remember that the logits at the current time step are tensors of size batch_size x vocabulary_size. For each data point in the batch (dim 0), the logits tensor assigns a score to each token in the vocabulary (dim 1); higher the token score means that that token is more likely to be the next token, compared to tokens with lower scores.

However, these raw scores need to be converted into a probability distribution that we can sample from. There are many ways of converting these logits into probability distributions and sampling from them, called decoding strategies. We will implement 4 different decoding strategies in this assignment, described below. Recall that we covered these decoding streategies in class (see lecture https://www.cs.cornell.edu/courses/cs4740/2025sp/lectures/Lec18.pdf) and also Jurafsky and Martin, Chapter 10.2 (https://web.stanford.edu/~jurafsky/slp3/10.pdf). We will implement 4 different decoding strategies in ``TODOS 4.1-1 - 4.1-4``.


But first, let us understand the overall workflow of generating text from the Seagull transformer. We'd like to draw your attention to the ``talk()`` function in the SeagullLM class. The function allows for our Seagull model to "speak". It takes as input the tensor of input_ids, hyperparameter num_samples that specifies how many outputs to generate for the given input (specified by input_ids), hyperparameter max_new_tokens that specifies the max generation length, and the decoding strategy (one of greedy_decode', 'sampling_decode', 'top_p_decode' or 'top_k_decode'.

Given these inputs, the talk function uses the Seagull model to produce up to max_new_tokens number of tokens that the model thinks is a likely continuation of the ongoing sequence (based on the context from the input input_ids).

Let's dig into the function a bit more. Here, we assume that the input_ids always contain one data sample, i.e. batch_size = 1. Given such a input_ids tensor, we create num_samples copies of this tensor. We do the same operation on position_ids. Therefore, all_input_ids and all_position_ids tensor is of size num_samples x seq_length. We treat each row of this "batch" as an independent data point during generation. This allows us to get different generated continuations for the same input.


Aside: You will note that if the sequence length (dim 1) of input_ids is greater than Seagull's _max_positions, we truncate the input from the left. Can you think of why we truncate from the left and not right?

What should be our stopping_condition for generation? We should stop generating when we either generate the max_new_tokens number of tokens. But also, recall that our processed data examples in training always end with the ``<|endofcaption|>`` token. We should also stop generating if all the data points in the batch have generated the ``<|endofcaption|>`` token.
To satisfy these criterion, the for loop iterates from 0 -> max_new_tokens, and the ``ended_mask`` tracks which of the generations in the batch have already generated the ``<|endofcaption|>`` token. ``ended_mask`` is initialized as a tensor of ``None``s of length num_samples.

<a name="sec41"></a>
#### [4.1] Decoding Strategies <small>[↩︎](#outline)</small>

To get started, we will implement our 4 decoding strategies. Note that you can implement these without worrying about the correctness of your copied code from HW3.

First, let's implement greedy decode, corresponding to <font color="orange">`TODO-4.1-1`</font> in `seagull/model/heads/seagull_lm.py`. The ``greedy_decode()`` takes the logits as input. For each datapoint, it selects the most probable token at each step. This is deterministic and always picks the token with the highest score.



Once you've implemented the function, you can use the following test case to test your implementation. But first, look at the example and try to convince yourself that the expected output is correct.

In [14]:
logits = torch.tensor([
    [1.0, 3.2, 0.5],
    [0.2, 0.1, 4.5],
    [-1.0, -0.5, -0.2]
])

expected = torch.tensor([
    [1],
    [2],
    [2]
])
output = greedy_decode(logits)
assert torch.equal(output, expected), f"Test failed. Got {output}, expected {expected}"

print("Test passed. Output:", output)

Test passed. Output: tensor([[1],
        [2],
        [2]])


Next, we will implement random sampling. Remember that logits provide you will raw scores. To sample, we first need to convert these logits into a probability distribution over the vocabulary tokens and sample from the resulting probability distribution. This allows us to introduce stochasticity; we can get distinct outputs from the model given the same input!

In this TODO, we will implement sampling with temperature scaling. Essentially, for temperature scaling, we divide the logits by a given temperature before converting to a probability distribution. Complete <font color="orange">`TODO-4.1-2`</font>, by implementing the  ``sampling_decode`` function that accepts the logits tensor and hyperparameter temperature as inputs.

Hint: You should use `torch.multinomial()` to draw a sample from a probability distribution.

In [15]:
output = sampling_decode(logits, temperature=0.9)

assert output.shape == expected.shape, f"Output tensor shape as expected. Does not guarantee correctness of code"

temps = [0.1, 0.9, 20, 50, 100]
for t in temps:
  output = sampling_decode(logits, temperature=t)
  print(output)


tensor([[1],
        [2],
        [2]])
tensor([[1],
        [2],
        [2]])
tensor([[2],
        [2],
        [2]])
tensor([[1],
        [0],
        [0]])
tensor([[2],
        [2],
        [1]])


Since ``sampling_decode`` is stochastic, the above only checks that your output tensors are the correct shape. Note that the expected shape is same for all decoding strategies.

<font color="orange">__Q2.__ Does the model generate less/more diverse samples when the temperature is high (say, 47.40)? What happens when the temperature is low (e.g., 0.1)?

</font>

__Answer.__

As shown with the printout above, when temperature is high, because all our logits will become small and also more similar, there's a greater chance for different types tokens to get sampled, but this includes insignificant ones. On the other hand, when temperature is low, temperature scaling will exagerrate logit differences. Therefore, we'll tend to get more greedy tokens.




Next, we will implement the next decoding strategy, i.e. top-k sampling. The basic idea is as follows: given an integer ``k``, top-k sampling limits the sampling space to the ``k`` most likely tokens, or the ``k`` highest scoring tokens in the vocabulary.

consider the example below. For the input "The sky is", we show the next token probabilities (not logits!) for 3 tokens (assume the next token probabilities of all others is 0 for this example).
The sky is
        blue 0.5
        yellow 0.2
        overcast 0.3

Say k=2. This means that we must only sample from the top-2 most likely tokens, i.e. blue and overcast, in our example.

Complete <font color="orange">`TODO-4.1-3`</font> by implementing the ``top_k_decode`` function. Note that you must also perform temperature scaling within the top_k_decode function. You can run the check below to ensure that your returned tensors are the right shape.

In [16]:
output = top_k_decode(logits, temperature=0.9, k=2)

assert output.shape == expected.shape, f"Output tensor shape as expected. Does not guarantee correctness of code"

---
<font color="orange"> Q3. What unintended consequences can the model's generative abilities have if the chosen k in top-k is too large?
</font>

__Answer.__

If the chosen k in top-k is too large, just like with the large temperature scaling issue, because your range of top values to choose from expands, you can riskk choosing a relatively insignificant token from that large range of top k values.

Finally, we will implement our last decoding strategy -- top-p sampling. Top-p sampling accepts a hyperparamter ``p``, which is a float between 0 and 1. Instead of choosing a fixed number of top tokens, top-p sampling samples from the smallest number of tokens, ordered from highest to lowest probabolility, whose cumulative probability exceeds this threshold p.  

Consider the example from above.
The sky is
        blue 0.5
        yellow 0.2
        overcast 0.3

Say p=0.6. The smallest set of of tokens that satisfy the above criterion is (blue, overcast); you must only sample from these tokens. Think how you will convert the now un-normalized probabilities (blue: 0.5, overcast: 0.3) to a probability distribution to sample from.

Remember that if p=0.2, the smallest set of of tokens that satisfy the above criterion will be (blue).

Complete <font color="orange">`TODO-4.1-4`</font> by implementing the ``top_p_decode`` function. Note that you must also perform temperature scaling within the top_k_decode function. You can run the check below to ensure that your returned tensors are the right shape.

In [17]:
output = top_p_decode(logits, temperature=0.9, p = 0.6)

assert output.shape == expected.shape, f"Output tensor shape as expected. Does not guarantee correctness of code"

### [3.2] Hey Seagull, tell me a joke! <small>[↩︎](#outline)</small>

> <font color="orange">File to be modified: `seagull/model/heads/seagull_lm.py`.</font>

Great! Now we have implemented all our decoding functions! We can now proceed to implementing the ``talk`` function of this assignment!

Note that we will use the same pre-trained model from HW3 to generate text. At this point, it would be good for you to ensure that all your function implementations from HW3 are correct. Although you can proceed without checking that, it will be easy for you to debug the ``talk()`` function. This is because, if the functions in HW3 are implemented correctly AND your ``talk`` function implementation is correct, the generated text will be reasonable.

We will use the same pre-trained Seagull transformer as HW3. Remember that the Seagull model's configuration is as follows:

* an embedding dimension of 768, divisible by 12, the number of heads,
* RoPE rotary encodings, instead of learned (or even absolute) positional embeddings,
*pre-normalization using RMS norm, as opposed to post-normalization using nn.LayerNorm, and
* the feed-forward net with: swish activation with GLU (SwiGLU) and a linear projection with intermediate dimension of 2,048, instead of two linear layers with intermediate dimension of 3,072.


This Seagull transformer, with 110.53 million trainable parameters (for comparison: GPT-2 small has 117 million parameters), was pre-trained on the OpenWebText dataset. First, let's load the model using the code below:

In [21]:
bbpe_tokenizer = BBPETokenizer()
bbpe_tokenizer.from_file(os.path.join(PRETRAINED_ARTEFACTS_DIR, "tokenizer"))
print(bbpe_tokenizer)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
with open(os.path.join(CONFIGS_DIR, "train_model.yml"), "r") as fp:
    config = yaml.safe_load(fp)
seagull_lm = SeagullLM(
    vocab_size=bbpe_tokenizer.vocab_size,
    padding_idx=bbpe_tokenizer.token2id(bbpe_tokenizer.pad_token),
    **config["model"],
).to(device)

pretrained_filename = "openwebtext_epochs=11_model.pt"
seagull_lm.from_pretrained(model_filepath=os.path.join(PRETRAINED_ARTEFACTS_DIR, f"model/{pretrained_filename}"))
print(f"model weights loaded using: {colored(pretrained_filename, 'blue')}")

BBPETokenizer(name=seagull-bbpe, vocab_size=33264, lowercase=False, punct_behavior=contiguous, special_tokens={'<|pad|>': 0, '<|unk|>': 1, '<|endoftext|>': 2, '<|scene|>': 3, '<|uncanny|>': 4, '<|caption|>': 5, '<|endofcaption|>': 6})
model weights loaded using: openwebtext_epochs=11_model.pt


Okay, let's get back to implementing the ``talk`` functionality which will generate text using the pre-trained SeagullLM transformer. First, let us complete the helper function `parse_before_eos`. This helper function is called within the ``talk`` function after the for loop. Remember that the for loop generates next tokens for the given input_ids (num_samples number of inputs) till the stopping criterion is satisfied, i.e. either ``max_new_tokens`` number of tokens are generated, or the ``<|endofcaption|>`` token is generated for each input.

The output of the for loop is all_input_ids, a tensor of size batch_size x seq_length. This means that inputs in the batch for which generation ended earlier (i.e. ``<|endofcaption|>`` already generated) will still contain generated tokens upto the length seq_length till ``<|endofcaption|>`` is generated for each input. The helper function `parse_before_eos` removes everything after the first ``<|endofcaption|>`` token in each sequence. It will be used after generation to trim extra tokens that follow the end of a caption.

Complete ``TODO-4.2 -1`` by implenting the `parse_before_eos` function. It wil take as input a  tensor of shape `(batch_size, sequence_length)` and the eos_token_id. We will use should be the token_id corresponding to token ``<|endofcaption|>``. Your function should return a `List[List[int]]` where each sequence is truncated after the first `eos_token_id`.

Run the following test case to make sure that your implementation is correct.

In [22]:
from seagull.model.heads.seagull_lm import parse_before_eos

eos_token_id = bbpe_tokenizer.token2id(END_OF_CAPTION_TOKEN)
all_input_ids = torch.Tensor([
    [3, 4, 6, 1, 5],
    [3, 4, 1, 5, 6],
    [3, 4, 1, 5, 29],
])

expected = [[3, 4, 6], [3, 4, 1, 5, 6], [3, 4, 1, 5, 29]]

output = parse_before_eos(all_input_ids, eos_token_id)
print(output)

assert output == expected, "Output of parse_before_eos as expected"

[[3.0, 4.0, 6.0], [3.0, 4.0, 1.0, 5.0, 6.0], [3.0, 4.0, 1.0, 5.0, 29.0]]


Finally, we will complete ``TODO 4.2-2`` that implements the for loop within the ``talk`` function in ``seagull_lm.py``. To remind oureselves, this function allows for our Seagull model to "speak". It takes as input a tensor of input_ids, hyperparameter num_samples that specifies how many outputs to sample, hyperparameter max_new_tokens, and the decoding strategy (one of greedy_decode', 'sampling_decode', 'top_p_decode' or 'top_k_decode').

Given these inputs, the talk function uses the Seagull model to produce up to max_new_tokens number of tokens that the model thinks is a likely continuation of the ongoing sequence (based on the context from the input input_ids).

Let's dig into the function a bit more. Here, we assume that the input_ids always contain one data sample, i.e. batch_size = 1. Given such a input_ids tensor, we create num_samples copies of this tensor. We do the same operation on position_ids. Our all_input_ids and all_position_ids tensor is of size num_samples x seq_length. We treat each row of this "batch" as an independant data point during generation. This allows us to get different generated continuations for the same input.

Within the for loop, the model performs a forward pass to get the logits for the next token prediction. This is already implemented for you. You need to implement the rest of the functionality within this for loop, particularly:

* use the given decoding strategy to sample the next token for all inputs in the batch.
* append the next token to the all_input_ids tensors to create the input for the next iteration of the loop. Do the same for all_position_ids.
* check if all inputs in the batch have already generated the eos_token_id. If yes, break the loop.

Once you implement your ``talk`` function, you can use the code below to generate text using the loaded pre-trained model.


In [23]:
prompt = "I was going to the store and"

max_new_tokens = 100
num_samples = 3
top_k = 5
temperature = 0.7
decoding_strategy="top_k_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py

make_seagull_talk(
    seagull_lm=seagull_lm,
    bbpe_tokenizer=bbpe_tokenizer,
    prompt=prompt,
    max_new_tokens=max_new_tokens,
    num_samples=num_samples,
    decoding_strategy="top_k_decode",
    decoding_kwargs={"temperature":temperature, "k":top_k},
)


['I was going to the store and I was going to buy my son’s shoes because it was a great idea.”\n\nThe company’s CEO, Jason Lewis, said that the company is “still in the early stages of the process” of making a new product.\n\nHe said that the company has been working hard to get the shoe ready and that it will be ready for the end of the year.\n\n“We’re very excited to be able to bring the shoe to market',
 'I was going to the store and I saw a guy in my room. I was like, \'What the fuck? I\'m going to get this guy.\'\n\n"I was like, \'You\'re going to have to take this guy out of this store.\'\n\n"I said, \'Oh my god, I can\'t do this.\' And then I went to the other room and he said, \'You can\'t do this. I can\'t do this,\'"',
 'I was going to the store and I saw a guy who was wearing a black shirt and a black shirt and I said, “You know, I’m not going to do this.” And I went, “No, I’m not going to do this.” I was like, I’m going to go and see what happens. I said, “You know, I’m not

If everything is implemneted correctly, the model's generations should be grammatical and reasonable. This informs us that our pretrained model is capable of understanding language, but what about humor? Let's choose a random sample from our dataset and see if the model can generate a humorous caption. (Feel free to change the sample index below.)

In [24]:
tokenized_dataset = datasets.load_from_disk(os.path.join(ARTEFACTS_DIR, "dataset/tokenized_dataset"))
tokenized_dataset

sample_idx = 100  # change as needed

# Choose from the test set to ensure captions are not included.
prompt = tokenized_dataset["test"][sample_idx]["text"]
print("\n".join(text_wrapper.wrap(prompt)))


<|scene|> A man is shaking violently. A waitress pours him another cup of coffee. A man eating a sandwich stares at them
at the diner. <|uncanny|> The man is shaking a lot due to too much coffee. <|caption|>


In [25]:
make_seagull_talk(
    seagull_lm=seagull_lm,
    bbpe_tokenizer=bbpe_tokenizer,
    prompt=prompt,
    max_new_tokens=max_new_tokens,
    num_samples=num_samples,
    decoding_strategy="top_k_decode",
    decoding_kwargs={"temperature":temperature, "k":top_k},
)

['<|scene|> A man is shaking violently. A waitress pours him another cup of coffee. A man eating a sandwich stares at them at the diner. <|uncanny|> The man is shaking a lot due to too much coffee. <|caption|>\n\n\n\n\nA man is shaking violently at the diner. A woman looks at him and says, "Hey, you\'re a woman." ."\n\nThe woman says, "Hey, you\'re a woman. I\'m a woman. I\'ve had enough. I\'ve had enough. I\'ve had enough. I\'ve had enough."\n\nThe woman says, "Hey, "Hey, you\'s okay,',
 '<|scene|> A man is shaking violently. A waitress pours him another cup of coffee. A man eating a sandwich stares at them at the diner. <|uncanny|> The man is shaking a lot due to too much coffee. <|caption|> A man is shaking violently at the restaurant. A woman walks past. ĐA man is shaking violently at the restaurant. ĐA man is shaking violently at the restaurant. A woman walks past. ĐA man is shaking violently at the restaurant. ĐA man is shaking violently at the restaurant. ĐA man is shaking viole

As expected, the model isn't picking up on the nature of the task, i.e., the objective of generating humorous captions; this could simply be because it is unlikely that the model is attuned to our template processing using control codes. Other than that, a few quick observations here:

* the model doesn't generate <|endoftext|> token anywhere in the 100 new tokens (think why this is unsurprising)—this is a potential problem for us for two reasons: 1) our task is to generate relatively short captions, and 2) our model has a maximum context length of 128;
* despite being trained on humor short texts, the model struggles with even understanding humor.
In the next subsection, we will proceed to finetune the pretrained model on our captions dataset, and analyze the finetuned model's generative capabilities.

<a name="sec5"></a>
### [5] Finetuning the Seagull to "understand" humor <small>[↩︎](#outline)</small>

Let us use ``train_seagull.py`` script to finetune the pretrained model. Given that the model already seems to have adequate language generation abilities, in theory, it should be sufficient to finetune the pretrained model for just 1-2 epochs.

Additional notes. The optimizer and learning rate hyper-hyperparameters are in ``scripts/configs/train_model.yml``, see keys: optimizer/finetuning and lr_scheduler/finetuning respectively; the training hyperparameters are in the same config file, see keys: trainer/finetuning and train_and_eval/finetuning. Feel free to experiment with any of these.

We provide a default setting of 16 batch size and 1 epoch for finetuning; feel free to adjust these as needed. Under this setting, it takes about ~22mins to finetune the model and ~3mins to run validation on a T4 (Colab-default) GPU.

In [26]:
batch_size = 16
num_epochs = 2

pretrained_filename = "openwebtext_epochs=11_model.pt"
pretrained_model_filepath = os.path.join(PRETRAINED_ARTEFACTS_DIR, f"model/{pretrained_filename}")
experiment_name = f"seagull_captions_batch-size={batch_size}_epochs={num_epochs}"

import os
os.environ["PYTHONPATH"] = "/content/drive/MyDrive/cs4740_programming_assignments/hw4-release" ##set this to the right path


In [27]:
!python3 scripts/train_model.py \
    --config-path={os.path.join(CONFIGS_DIR, "train_model.yml")} \
    --basepath-to-tokenized-dataset={os.path.join(ARTEFACTS_DIR, "dataset/tokenized_dataset")} \
    --tokenizer-basepath={os.path.join(PRETRAINED_ARTEFACTS_DIR, "tokenizer")} \
    --basepath-to-store-results={os.path.join(ARTEFACTS_DIR, "experiments")} \
    --batch-size={batch_size} \
    --num-epochs={num_epochs} \
    --pretrained-checkpoint-or-model-filepath={pretrained_model_filepath} \
    --experiment-name={experiment_name}


+-----------------------------------------------------------+------------+---------------+
|                           module                          | num_params | requires_grad |
+-----------------------------------------------------------+------------+---------------+
|          seagull.embedding.token_embedding.weight         |  25546752  |      True     |
|   seagull.transformer_layers.0.mha.qkv_transform.weight   |  1769472   |      True     |
|    seagull.transformer_layers.0.mha.qkv_transform.bias    |    2304    |      True     |
|  seagull.transformer_layers.0.mha.output_transform.weight |   589824   |      True     |
|   seagull.transformer_layers.0.mha.output_transform.bias  |    768     |      True     |
|        seagull.transformer_layers.0.ffn.glu.weight        |  3145728   |      True     |
|       seagull.transformer_layers.0.ffn.linear.weight      |  1572864   |      True     |
|        seagull.transformer_layers.0.layer_norm.gain       |    768     |      True     |

Once the model finetuning completes running, load the best model (or the only model) based on the observed validation performance measured through loss or perplexity. Set the best_epoch below to reflect the epoch (zero-indexed) that resulted in the best model.

Tip. Step and epoch-level metrics for both training/finetuning and validation, and model checkpoints are written to artefacts/experiments/<experiment-name>. Also, if you wish to continue finetuning from an already finetuned checkpoint, modify the value of --pretrained-checkpoint-or-model-filepath in the above command above accordingly.

In [28]:
# Change the best epoch value.
best_epoch = 0
bbpe_tokenizer = BBPETokenizer()
bbpe_tokenizer.from_file(os.path.join(PRETRAINED_ARTEFACTS_DIR, "tokenizer"))

checkpoint_path = f"seagull_captions_batch-size={batch_size}_epochs={num_epochs}/checkpoints/checkpoint_{best_epoch}.ckpt"
print(f"loading model from: {colored(checkpoint_path, 'blue')}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
with open(os.path.join(CONFIGS_DIR, "train_model.yml"), "r") as fp:
    config = yaml.safe_load(fp)
seagull_lm = SeagullLM(
    vocab_size=bbpe_tokenizer.vocab_size,
    padding_idx=bbpe_tokenizer.token2id(bbpe_tokenizer.pad_token),
    **config["model"],
).to(device)
seagull_lm.from_pretrained_or_checkpoint(os.path.join(ARTEFACTS_DIR, f"experiments/{checkpoint_path}"))

seagull_lm.print_params()


loading model from: seagull_captions_batch-size=16_epochs=2/checkpoints/checkpoint_0.ckpt
+-----------------------------------------------------------+------------+---------------+
|                           module                          | num_params | requires_grad |
+-----------------------------------------------------------+------------+---------------+
|          seagull.embedding.token_embedding.weight         |  25546752  |      True     |
|   seagull.transformer_layers.0.mha.qkv_transform.weight   |  1769472   |      True     |
|    seagull.transformer_layers.0.mha.qkv_transform.bias    |    2304    |      True     |
|  seagull.transformer_layers.0.mha.output_transform.weight |   589824   |      True     |
|   seagull.transformer_layers.0.mha.output_transform.bias  |    768     |      True     |
|        seagull.transformer_layers.0.ffn.glu.weight        |  3145728   |      True     |
|       seagull.transformer_layers.0.ffn.linear.weight      |  1572864   |      True     |


Let's now check if the finetuned model has unlocked the superpower of contextual humor, in that, is the model now capable of generating humorous captions for a given scene and uncanny description.

Same as before, let's load a random test sample and see how the model captions it. Change the sample_idx below to load a different sample and the cell below that stores the values for generation hyperparameters (e.g., temperature, num_samples), you're free to change those too.



In [29]:
sample_idx = 10  # change as needed

# Choose from the test set to ensure captions are not included.
prompt = tokenized_dataset["test"][sample_idx]["text"]
print("\n".join(text_wrapper.wrap(prompt)))


<|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking
out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|>


In [30]:
max_new_tokens = 100
num_samples = 3
top_k = 5
temperature = 0.7
decoding_strategy="top_k_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py

make_seagull_talk(
    seagull_lm=seagull_lm,
    bbpe_tokenizer=bbpe_tokenizer,
    prompt=prompt,
    max_new_tokens=max_new_tokens,
    num_samples=num_samples,
    decoding_strategy="top_k_decode",
    decoding_kwargs={"temperature":temperature, "k":top_k},
)

["<|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|> It's a good idea, I'm just going to have to get the dog out of there, you know? <|endofcaption|>",
 "<|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|> I'm afraid I'll be in the wrong place at the wrong time. <|endofcaption|>",
 "<|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|> I'm afraid you're not sure I'm the only one who's still in it. <|endofcaption|>"]

In [32]:
#USING EXAMPLE CODE
sample_idxs = [10, 25, 42]

for idx in sample_idxs:
    prompt = tokenized_dataset["test"][idx]["text"]

    max_new_tokens   = 100
    num_samples      = 3
    temperature      = 0.7
    top_k            = 5
    decoding_strategy = "top_k_decode"

    # sample
    captions = make_seagull_talk(
        seagull_lm=seagull_lm,
        bbpe_tokenizer=bbpe_tokenizer,
        prompt=prompt,
        max_new_tokens=max_new_tokens,
        num_samples=num_samples,
        decoding_strategy=decoding_strategy,
        decoding_kwargs={"temperature": temperature, "k": top_k},
    )

    print("Sample ", prompt, "\n")
    i = 1
    for cap in captions:
        print(i, cap)
        i += 1

Sample  <|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|> 

1 <|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|> It's a good idea, I'm just going to have to get the dog out of there, you know? <|endofcaption|>
2 <|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad. <|uncanny|> There is a centaur there and a man with a horse head. Both are not real. <|caption|> I'm afraid I'll be in the wrong place at the wrong time. <|endofcaption|>
3 <|scene|> A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking o

Okay, has the model improved in its humor understanding? Is it able to generate context-specific, meaningful captions? We've also had the problem of the model not generating an <|endofcaption|> or <|endoftext|> token—has this been resolved with finetuning?

Note. The function make_seagull_talk() stops generating further after the generation of an <|endofcaption|> token; so if the above captions appear to be shorter than the specified max_new_tokens, then the model has learned to associate captions as being short-length sequences ending in an <|endofcaption|> token.



<font color="orange">Q4: Using your finetuned model, generate captions for some random test samples from the captions dataset and note down two/three of the most humorous ones below; format: scene, uncanny description, and the model-generated caption. (Maximum score: 2 points.)

This question is intended to have you inspect the generative and humor understanding abilities of the model, and to understand the impact of generation hyperparameters (e.g., temperature, top_k) on the generation diversity/quality.

</font>

__Answer.__

Example 1:
Scene: A centaur man is looking at a naked man with an upper torso of a horse. The horse man with his butt sticking out looks sad
Uncanny description: There is a centaur there and a man with a horse head. Both are not real.
Model-generated caption: I'm afraid I'll be in the wrong place at the wrong time.


Example 2:
Scene:  A couple is at a car lot when a salesman shows them a hideous car. the care has cat arms coming out of the tires and cat eyes for headlights and razor sharp teeth.
Uncanny description: Cars do not have cat body parts coming out of it.
Model-generated caption: I'm just saying, I'm not afraid of the car.

Example 3:
Scene: Two men are in a courtroom. A judge is addressing a bailiff.
Uncanny description:  Nothing is out of place that I can see.
Model-generated caption: It's a good way to settle the case.

<a name="sec6"></a>
### [6] Model evaluation <small>[↩︎](#outline)</small>

The main motivation and goal of this assignment was to build your own transformer, a humorous one at that!

We will evaluate our models using the ROUGE score between the generated captions and the gold captions in the dataset.

Use the following script to generate outputs for 100 examples in the validation set.


In [33]:
max_new_tokens = 100
num_samples = 1
top_k = 5
temperature = 2
decoding_strategy="top_k_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="top_k_decode",
                    decoding_kwargs={"temperature":temperature, "k":top_k},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])


 It'm your Honor, it's mine! <|endofcaption|>
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>


You can quantify how good these generations are using the following code block that computes ROUGE scores.

In [40]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print(avg_scores)



{'rouge1': 0.1419763668208131, 'rouge2': 0.0030050125313283212}


In [43]:
#USING FORMAT FROM ABOVE
max_new_tokens = 100
num_samples = 1
top_k = 20
temperature = 0.7
decoding_strategy="top_k_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="top_k_decode",
                    decoding_kwargs={"temperature":temperature, "k":top_k},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print(avg_scores)


 Let's see how you perform. <|endofcaption|>
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>
{'rouge1': 0.14216684301128932, 'rouge2': 0.0030050125313283212}


In [45]:
#USING FORMAT FROM ABOVE
max_new_tokens = 100
num_samples = 1
top_k = 20
temperature = 7
decoding_strategy="top_k_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="top_k_decode",
                    decoding_kwargs={"temperature":temperature, "k":top_k},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print(avg_scores)


 Let you look. There were very few men ... but one person (Malinowitz, the real winner at an early court contest), with the full consent on the defense part !! It will have never happen....And this trial. All you see you say will been “No”? And the winner? We are now a part owner of BetoCon and all our stock, our stock has crashed to a tiny 4-bit high !! That’ve never looked
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>
{'rouge1': 0.06665912860270261, 'rouge2': 0.001431158071299182}


In [48]:
#USING FORMAT FROM ABOVE
max_new_tokens = 100
num_samples = 1
decoding_strategy="greedy_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="greedy_decode",
                    decoding_kwargs={},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print("greedy decode", avg_scores)




 I'm sorry, but I'm not sure you're the only one who's been in the wrong place. <|endofcaption|>
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>
greedy decode {'rouge1': 0.15717696361710565, 'rouge2': 0.01129175776251948}


In [ ]:
#USING FORMAT FROM ABOVE
#TAkING TOO LONG TO RUN SO STOPPED IT -> Decided to no longer test top_p_decode
max_new_tokens = 100
num_samples = 1
p = 0.01
temperature = 0.7
decoding_strategy="top_p_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="top_p_decode",
                    decoding_kwargs={"temperature": 0.7, "p": 0.8},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print(" p decode", avg_scores)




 Let's see what happens in the case of the defendant. <|endofcaption|>
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>
 p decode {'rouge1': 0.16383369258151248, 'rouge2': 0.006172490832081625}


In [ ]:
#USING FORMAT FROM ABOVE
#TAkING TOO LONG TO RUN SO STOPPED IT -> Decided to no longer test top_p_decode
max_new_tokens = 100
num_samples = 1
p = 0.1
temperature = 0.7
decoding_strategy="top_p_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="top_p_decode",
                    decoding_kwargs={"temperature": 0.7, "p": 0.8},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print(" p decode", avg_scores)




In [44]:
#USING FORMAT FROM ABOVE
max_new_tokens = 100
num_samples = 1
temperature = 0.1
decoding_strategy="sampling_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="sampling_decode",
                    decoding_kwargs={"temperature": 0.7},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print("Sampling decode", avg_scores)




 Let's see how late it's all been, sir. <|endofcaption|>
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>
Sampling decode {'rouge1': 0.1250749896073683, 'rouge2': 0.009582989190406071}


In [46]:
#USING FORMAT FROM ABOVE
max_new_tokens = 100
num_samples = 1
temperature = 1
decoding_strategy="sampling_decode" # choose from the keys in DECODING_FUNCS in seagull_lm.py


generated_outputs = []
gold_outputs = []
for i in range(100):
  text_curr = processed_dataset['val'][i]['text']
  input_curr = text_curr.split('<|caption|>')[0] + '<|caption|>'
  caption_curr = text_curr.split('<|caption|>')[1]
  gold_outputs.append(caption_curr)
  generated_output_curr = make_seagull_talk(
                    seagull_lm=seagull_lm,
                    bbpe_tokenizer=bbpe_tokenizer,
                    prompt=prompt,
                    max_new_tokens=max_new_tokens,
                    num_samples=num_samples,
                    decoding_strategy="sampling_decode",
                    decoding_kwargs={"temperature": 1},
                  )
  generated_outputs.append(generated_output_curr[0].split('<|caption|>')[1])

print(generated_outputs[0])
print(gold_outputs[0])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

scores = {'rouge1': [], 'rouge2': []}
for pred, ref in zip(generated_outputs, gold_outputs):
    result = scorer.score(ref, pred)
    for key in scores:
        scores[key].append(result[key].fmeasure)

avg_scores = {key: np.mean(scores[key]) for key in scores}
print("Sampling decode", avg_scores)




 Let's settle this because it's all been going on for an additional 10 days! <|endofcaption|>
 Our estate planner wants to know if you would like your inheritance in a lump sum or spread out over nine lives. <|endofcaption|>
Sampling decode {'rouge1': 0.11787971846292551, 'rouge2': 0.010242632701137318}


You should try to identify the decoding parameter set that gives you the highest rouge score.

You can experiment with the following hyperparameters: top_p, temperature if using top_p sampling, top_k, temperature if using top_k sampling, or just the temperature if using random sampling.

<font color="orange"> Q5: Use your best model checkpoint to report ROUGE1 and ROUGE 2 scores for the 100 validation examples. <br/><br/> You should report scores for each of the 4 decoding strategies. For random sampling, report results for 2 different values of temperature.
For top_p, and top_k, report results using 2 different the values p and k each. <br/><br/>
Which decoding strategy and hyperparameter set gives you the best performance?

</font>

__Answer.__

The decoding strategy and hyperparameter that gives the strongest performance is greedy decode ran with 1 sample and max_tokens = 100, out of all experiments tested above.

<font color="orange"> Q6: Comment on whether you believe ROUGE is a good metric for this task. Why or why not? Discuss pros and cons of alaternative text generation evaluation metrics we discussed in class.

</font>

__Answer.__

We learned about rouge in context of n-grams, calculating word overlaps, which may not be the most contextually relevant for our goal of generating humorous captions. Also, since we can have different correct captions for our inputs, rouge may rank one correct caption above one another correct caption even though they're both valid. The advantage of ROUGE is that it's quite easy to calculate.

It's difficult to account for different "equally" correct captions. Even with QA techniques we discussed in class, it still focuses more on factuality assessment than humorous assessment. I think to assess semantic/humor, perhaps we could also build a separate transformer model to classify how humorous our caption is on a scale from 1 to 10, so one model for assessing factuality relevance and one model for assessing humor.

<a name="sec7"></a>
### [7] Final Submission <small>[↩︎](#outline)</small>

Yay! Now that we've successfully finetuned our Seagull to generate short, humorous captions, let's bundle everything that's needed to make a submission on the submission site(s).

For your final submission, you will need to submit the hw4_submission.zip and a .pdf of this notebook file (or a separate file containing answers to your written questions; don't forget to attribute!) on the submission site(s). Note: this notebook will only be used to grade your answers to the written questions; you will not be graded on any code in this notebook file.


Running the cell below generates a hw4_submission.zip file in the artefacts folder. Caution: the script will overwrite files with the same name existing in the artefacts folder.



In [47]:
!python3 -m scripts.make_submission \
    --basepath-to-store-submission={os.path.join(ARTEFACTS_DIR, submission_filepath)} \
    --net-ids={net_ids}

if os.path.isfile(f"{os.path.join(ARTEFACTS_DIR, 'hw4_submission.zip')}"):
    display(success())
else:
    print(colored("Oops, something went wrong!", "red"))

submission stored at: /content/drive/MyDrive/cs4740_programming_assignments/hw4-release/artefacts/hw4_submission.zip
Success!
